In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler 
from sklearn import preprocessing
from sklearn import metrics

In [2]:
df=pd.read_csv('finaldata.csv')

In [3]:
df.columns

Index(['sex', 'Predicted_age', 'Weekday', 'TimeOfDay', 'mean', 'StressLevel'], dtype='object')

In [4]:
df.head()

,sex,Predicted_age,Weekday,TimeOfDay,mean,StressLevel
0,F,18,Saturday,Night,65.0,Very Stressed
1,F,18,Thursday,Eve,65.0,Very Stressed
2,F,19,Thursday,Eve,65.0,Very Stressed
3,F,19,Thursday,Night,10.0,Great
4,F,19,Wednesday,Night,52.0,Stressed


In [5]:
#onehotencode

from sklearn.preprocessing import LabelEncoder
# creating initial dataframe

labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
df['encWeekday'] = labelencoder.fit_transform(df['Weekday'])
df['encTimeOfDay'] = labelencoder.fit_transform(df['TimeOfDay'])
df['encsex'] = labelencoder.fit_transform(df['sex'])
 

In [6]:
#scale age
df['Predicted_age'] = StandardScaler().fit_transform(np.array(df['Predicted_age']).reshape(-1, 1))

In [7]:
#SMOTE oversampling to handle imbalanced class
X = df[['Predicted_age','encWeekday','encTimeOfDay', 'encsex']]
y = df['StressLevel']

from sklearn.model_selection import train_test_split 
  
# split into 70:30 ration 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0) 

from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state = 2) 
X_res, y_res = sm.fit_sample(X_train, y_train.ravel()) 
df = pd.concat([pd.DataFrame(X_res), pd.DataFrame(y_res)], axis=1)
df.columns = ['Predicted_age','encWeekday','encTimeOfDay', 'encsex','StressLevel']

  

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/loca

In [8]:
df.groupby('StressLevel').count()

,Predicted_age,encWeekday,encTimeOfDay,encsex
StressLevel,,,,
Alright,396,396,396,396
Exhausted,396,396,396,396
Great,396,396,396,396
Stressed,396,396,396,396
Very Stressed,396,396,396,396


In [10]:
#modeltraining
X = df[['Predicted_age','encWeekday','encTimeOfDay', 'encsex']]
y = df['StressLevel']
from sklearn.model_selection import train_test_split 
  
# split into 70:30 ration 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0) 
from sklearn.ensemble import RandomForestClassifier
#randomforest

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7087542087542088


In [ ]:
#tuning hyperParameters
'''
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

 
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

model_para = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_model = RandomForestClassifier(random_state=1)

# set up grid search meta-estimator
clf = GridSearchCV(rf_model, model_params, cv=5)

# train the grid search meta-estimator to find the best model
model = clf.fit(X_train, y_train)

# print winning set of hyperparameters
from pprint import pprint
pprint(model.best_estimator_.get_params())
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test,y_pred))

''''

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 6,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 150,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 1,
 'verbose': 0,
 'warm_start': False}
Accuracy: 0.7112135176651305


In [11]:
# 5. Declare data preprocessing steps
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

 

model_params = {'n_estimators': [150],
               'max_features': ['sqrt'],
               'min_samples_split': [6],
               'min_samples_leaf': [1],
               'bootstrap': [True]}

rf_model = RandomForestClassifier(random_state=1)

# set up grid search meta-estimator
clf = GridSearchCV(rf_model, model_params, cv=5)

# train the grid search meta-estimator to find the best model
model = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test,y_pred))


Accuracy: 0.7205387205387206
